In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

In [ ]:
data=pd.read_csv('../input/mnist-in-csv/mnist_train.csv')

In [ ]:
test_data=data=pd.read_csv('../input/mnist-in-csv/mnist_test.csv')

In [ ]:
test_data.head()

In [ ]:
data.head()

In [ ]:
digit=data['label']
digit_mat=data.drop('label',axis=1)
test_digit=test_data['label']
test_digit_mat=test_data.drop('label',axis=1)

In [ ]:
digits=min_max_scaler.fit_transform(np.array(digit_mat))
test_digits=min_max_scaler.fit_transform(np.array(test_digit_mat))

In [ ]:
first_digit=digits[0,:].reshape(28,28)
test_first_digit=test_digits[0,:].reshape(28,28)

In [ ]:
import matplotlib


In [ ]:
 matplotlib.pyplot.imshow(first_digit) # same as first label in train data

In [ ]:
 matplotlib.pyplot.imshow(test_first_digit) # same as first label in test data

#### The notebook is for building a model for multiclassification model for digit classification.

In [ ]:
pd.Categorical(test_digit)

In [ ]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
digit_tensors=to_categorical(digit,10)
test_digit_tensors=to_categorical(digit,10)

In [ ]:
from torch.utils.data import Dataset
from torchvision import datasets
import torchvision

In [ ]:

#digits

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, target, features):
        self.labels = target
        self.features =features
       

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        
        value = self.features[idx]
        label = self.labels[idx]
        
        return value, label


In [ ]:
train_data=CustomDataset(digit_tensors,torch.tensor(digits))
test_data=CustomDataset(test_digit_tensors,torch.tensor(test_digits))

In [ ]:
data_batched=torch.utils.data.DataLoader(train_data, batch_size=600, shuffle=False)
data_batched_test=torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=False)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#weights
#model.load_state_dict(torch.load('model_weights.pth'))
#model.eval()

In [ ]:
class cnn(nn.Module):
    
    def __init__(self,num_features,Batch_size):
        super().__init__()
        self.linear=nn.Linear(num_features,100)
        self.linear2=nn.Linear(100,10)
        self.dropout = nn.Dropout(p=0.2)
        self.batchnorm1 = nn.BatchNorm1d(10)
        
        
    def forward(self,x):
        x=self.linear(x)
        x=F.relu(x)
        x=self.linear2(x)
        x=F.sigmoid(x)
        return x

In [ ]:
model=cnn(784,600)

In [ ]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# train a model

In [ ]:
for j in range(4):
    for epoch in range(100):  # loop over the dataset multiple times
        running_loss = 0.0
        for i, data in enumerate(data_batched, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs,labels = data

            # zero the parameter gradients
            optimizer.zero_grad()
            # forward + backward + optimize
            outputs = model.forward(x=inputs.float())
            #print(outputs)
            loss = criterion(outputs.reshape(labels.shape), labels.float())
            loss.backward()
            optimizer.step()
            # print statistics
            running_loss += loss.item()
            if i % 100 == 0:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

print('Finished Training')

In [ ]:
acc=sum([int(torch.argmax(model(torch.tensor(digits[i
                                          ]).float())))==digit[i] for i in range(len(digit))])/len(digit)

In [ ]:
print('The accuraccy is ',acc,'%')

In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')